Forca Otimizada

Esse código cria um jogador de forca através dos conceitos de frequência de palavras e árvore de escolha, vistos no sexto e sétimo módulos do curso.

O código abaixo já foi entregue pronto no enunciado da APS e refere-se à implementação de um jogo de forca, com funções definidas para iniciar um jogo, tentar uma letra ou palavra.

In [64]:
import random
class JogoDeForca:
    def __init__(self):
        import requests
        url = 'https://www.ime.usp.br/~pf/dicios/br-sem-acentos.txt'
        r = requests.get(url, allow_redirects=True)
        if r.status_code==200:
            self.content = str(r.content.decode()).split('\n')
        else:
            print("Erro: ", r.status_code)
    
    def novo_jogo(self, vidas=5):
        self.vidas = vidas
        self.palavra = random.choice(self.content)
        return len(self.palavra)

    def tentar_letra(self, letra):
        if self.vidas > 0:
            if letra in self.palavra:
                return [idx for idx in range(len(self.palavra)) if self.palavra[idx]==letra]
            else:
                self.vidas -= 1
                if self.vidas == 0:
                    print("Fim de jogo!")
                    return False
                else:
                    return []
        
    def tentar_palavra(self, palavra):
        if self.vidas > 0:
            if self.palavra == palavra:
                print ("Ganhou!")
                return True
            else:
                self.vidas = 0
                print("Fim de jogo!")
                return False

A primeira parte para criar um agente jogador de forca é criar um critério para os "chutes" das letras. Para isso, são percorridas as letras das palavras presentes no dicionário, e se elas já não tiverem sido chutadas, são adicionadas num dicionário "letras" com suas respectivas frequências. Posteriormente, são calculadas as probabilidades dessas letras nas palavras fornecidas, e a letra com mais probabilidade é chutada. Esse processo envolve o conceito de probabilidades de símbolos de string, visto na última aula, que é parte integrante de um algoritmo de Huffman.

In [65]:
def letra_a_chutar(palavras, letras_chutadas):
    total_letras = 0
    letras = {}
    for palavra in palavras:
        for letra in palavra:
            if letra not in letras_chutadas:
                total_letras += 1
                if letra in letras:
                    letras[letra] += 1
                else:
                   letras[letra] = 1

    for letra in letras:
        letras[letra] = letras[letra]/total_letras

    letra = max(letras, key=letras.get)
    return letra

O próximo passo é encontrar um critério para chutar uma palavra. Parte do código de chutar uma letra é reaproveitado, pois novamente é construído um dicionário com probabilidades de letras. Em seguida, é formada uma espécie de árvore de decisão para escolher a palavra a ser chutada, conceito visto nos módulos 6 (junto de classificadores lineares) e 7 (quando estudado o funcionamento do código de Huffman). O primeiro desdobramento é um filtro, que capta apenas as palavras do dicionário com tamanho igual à palavra fornecida pelo jogo. Um dicionário com as probabilidades das letras dentro das palavras filtradas é criado, e depois o dicionário das palavras filtradas é ordenado, de forma que as palavras que contenham letras de maior probabilidade fiquem primeiro. Por último, são mantidas apenas as palavras que contém a letra atual da palavra misteriosa na mesma posição.

In [66]:
def palavra_a_chutar(palavras, palavra_misteriosa, letras_chutadas):
    
    palavras_filtradas = [p for p in palavras if len(p) == len(palavra_misteriosa)]

    total_letras = len(palavras_filtradas) 
    letras = {}
    for palavra in palavras_filtradas:
        for letra in palavra:
            if letra not in letras_chutadas:
                if letra in letras:
                    letras[letra] += 1
                else:
                    letras[letra] = 1
    
    for letra in letras:
        letras[letra] = letras[letra] / total_letras

    palavras_filtradas.sort(key=lambda p: sum(letras.get(letra, 0) for letra in p), reverse=True)
    
    for i, letra in enumerate(palavra_misteriosa):
        if letra != '_':
            palavras_filtradas = [p for p in palavras_filtradas if p[i] == letra]
    
    return palavras_filtradas[0]




Aqui é criado um loop que simula o jogo de forca por 100 vezes, com o nosso agente. O critério para entrar no loop é que a quantidade de vidas seja maior que 1, pois é necessário ter pelo menos uma vida para testar a palavra a escolhida, e essas vidas serão sendo decrescidas conforme as letras forem sendo chutadas, e talvez erradas. Sempre que entra no loop, uma letra é chutada, e se ela existir na palavra, é revelada. Quando restar apenas uma vida ou todas as letras tiverem sido reveladas, o jogador pode escolher uma palavra, e ela é comparada com a palavra misteriosa. 

No final, é impressa a quantidade de erros e acertos, e no caso dos erros, qual era a palavra chutada e qual era a palavra certa. Pode-se perceber que quando o agente erra, são palavras muito parecidas com a palavra em questão, então pode-se dizer que o jogador é bem-sucedido na maioria dos casos. Essa falha ocorre porque em certos casos, existem palavras muito parecidas, que mesmo lidando com as probabilidades de cada letra, é difícil afirmar com precisão qual é a palavra correta.

In [68]:
erros = 0
acertos = 0

for _ in range(100):
    novo_jogo = JogoDeForca()
    novo_jogo.novo_jogo()
    palavra_misteriosa = ["_" for _ in range(len(novo_jogo.palavra))]
    letras_chutadas = []
    

    while novo_jogo.vidas > 1:
        letra = letra_a_chutar(novo_jogo.content, letras_chutadas)
        letras_chutadas.append(letra)
        tentativa_letra = novo_jogo.tentar_letra(letra)

        if tentativa_letra != False:
            if len(tentativa_letra) > 0:
                for i in tentativa_letra:
                    palavra_misteriosa[i] = letra
        


        if novo_jogo.vidas == 1 or "_" not in palavra_misteriosa:
            palavra_escolhida = palavra_a_chutar(novo_jogo.content, palavra_misteriosa, letras_chutadas)    
            tentativa_palavra = novo_jogo.tentar_palavra(palavra_escolhida)

        

    if palavra_escolhida != novo_jogo.palavra:
        print("A palavra era: ", novo_jogo.palavra)
        print("A palavra chutada foi: ", palavra_escolhida)
        erros +=1
    else:
        acertos +=1

print("Erros:", erros)
print("Acertos:", acertos)

Ganhou!
Ganhou!
Ganhou!
Fim de jogo!
A palavra era:  entabulasses
A palavra chutada foi:  enganchasses
Ganhou!
Ganhou!
Ganhou!
Ganhou!
Ganhou!
Ganhou!
Ganhou!
Ganhou!
Ganhou!
Ganhou!
Ganhou!
Ganhou!
Ganhou!
Ganhou!
Ganhou!
Ganhou!
Ganhou!
Ganhou!
Ganhou!
Ganhou!
Ganhou!
Ganhou!
Ganhou!
Ganhou!
Ganhou!
Ganhou!
Ganhou!
Ganhou!
Ganhou!
Ganhou!
Fim de jogo!
A palavra era:  esparzes
A palavra chutada foi:  estardes
Ganhou!
Ganhou!
Ganhou!
Ganhou!
Ganhou!
Ganhou!
Ganhou!
Ganhou!
Ganhou!
Ganhou!
Ganhou!
Ganhou!
Ganhou!
Fim de jogo!
A palavra era:  sonhar
A palavra chutada foi:  sondar
Ganhou!
Ganhou!
Ganhou!
Ganhou!
Ganhou!
Ganhou!
Ganhou!
Ganhou!
Ganhou!
Ganhou!
Fim de jogo!
A palavra era:  quinto
A palavra chutada foi:  ionico
Fim de jogo!
A palavra era:  sorvidas
A palavra chutada foi:  sordidas
Fim de jogo!
A palavra era:  urtiga
A palavra chutada foi:  druida
Ganhou!
Ganhou!
Fim de jogo!
A palavra era:  vivificaria
A palavra chutada foi:  divinizaria
Ganhou!
Ganhou!
Ganhou!
Fim de jogo!
